<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/CNN/NN_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Classification by Neural Network and word Embedding**

----------

 import library

  إستيراد المكتبات

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import nltk
nltk.download('stopwords')
from nltk.stem.isri import ISRIStemmer
import re
import string

from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras import layers
from sklearn.preprocessing import LabelEncoder
seed = 42
np.random.seed(seed)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


--------

**1- Import Data set**

استيراد ملفات البيانات


In [4]:
# dataset = pd.read_excel("drive/MyDrive/mining/1.xlsx")
# dataset = pd.read_excel("drive/MyDrive/mining/2.xlsx")
# dataset = pd.read_excel("drive/MyDrive/mining/3.xlsx")
dataset = pd.read_excel("drive/MyDrive/mining/4.xlsx")

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68690 entries, 0 to 68689
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    68312 non-null  object
 1   targe   68690 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


------
shuffle  
بعثرة البيانات

In [5]:
dataset=shuffle(dataset)
dataset

,text,targe
18590,بجمال فريد وشخصية قوية متعددة المواهب استطاعت ...,0
27642,أخبارنا المغربية متابعة حضر ثلاثة مستشارين فقط...,3
7875,عبدالاله بوسحابة اخبارنا المغربية تعقد الهيئة ...,2
66249,في ما يلي نتائج عملية سحب قرعة نهائيات كأس الع...,4
36467,ندد عدد من الفنانين الإيطاليين والمغاربة بطريق...,0
...,...,...
37194,تمكنت عناصر الدرك الملكي من توقيف مرتكب جريمة ...,1
6265,تمكنت عناصر الجمارك المشتغلة بالمعبر الحدودي ب...,1
54886,اهتزت القنيطرة نهاية الأسبوع الماضي على وقع فض...,1
860,أخبارنا فهـــد الباهـــي عاد المدون والبودكاس ...,0


------
**2- Clean Data**

تنظيف البيانات

In [6]:
print(np.sum(dataset.isnull().any(axis=1)))
dataset= dataset.dropna()
print(np.sum(dataset.isnull().any(axis=1)))

#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
           
remove_hashtag(dataset)
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc
    
stemmer = ISRIStemmer()
dataset["text"] = dataset['text'].apply(lambda x: processDocument(x, stemmer))
print(dataset)

reviews = dataset['text'].values
t = Tokenizer(oov_token='<UNK>')
# fit the tokenizer on the documents
t.fit_on_texts(reviews)
t.word_index['<PAD>'] = 0

VOCAB_SIZE = len(t.word_index)

378
0
                                                    text  targe
18590  بجمال فريد وشخصيه قويه متعده المواهب استطاعت ا...      0
27642  اخبارنا المغربيه متابعه حضر ثلاثه مستشارين فقط...      3
7875   عبدالاله بوسحابه اخبارنا المغربيه تعقد الهيئه ...      2
66249  في ما يلي نتائج عمليه سحب قرعه نهائيات كاس الع...      4
36467  ند عد من الفنانين الايطالين والمغاربه بطريقه ا...      0
...                                                  ...    ...
37194  تمكنت عناصر الدرك الملكي من توقيف مرتكب جريمه ...      1
6265   تمكنت عناصر الجمارك المشتغله بالمعبر الحدودي ب...      1
54886  اهتزت القنيطره نهايه الاسبوع الماضي علي وقع فض...      1
860    اخبارنا فهد الباهي عاد المدون والبودكاس المغرب...      0
15795  تبدا يوم الثلاثاء يونيو حزيران وقائع محاكمه لا...      4

[68312 rows x 2 columns]


---------
Tokenizer function


In [7]:
def Tokenizer_UNK(train_x,test_reviews,train_y, test_y):
  norm_train_reviews = (train_x)
  norm_test_reviews = (test_reviews)

  le = LabelEncoder()
  y_train = le.fit_transform(train_y)
  y_test = le.transform(test_y)

  from keras.utils.np_utils import to_categorical
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)
  train_sequences = t.texts_to_sequences(norm_train_reviews)
  test_sequences = t.texts_to_sequences(norm_test_reviews)
  print("Vocabulary size={}".format(len(t.word_index)))
  print("Number of Documents={}".format(t.document_count))

  MAX_SEQUENCE_LENGTH = 1000
  # pad dataset to a maximum review length in words
  X_train = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
  X_test = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
  # X_train.shape, X_test.shape
  return X_train, X_test ,y_train,y_test

---------
model NN function

عمل تابع لبناء موديل الشبكة العصبية 

In [8]:
def set_model():
  VOCAB_SIZE = len(t.word_index)
  MAX_SEQUENCE_LENGTH = 1000
  EMBED_SIZE = 300
  
  # create the model
  global model
  model = Sequential()
  model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=MAX_SEQUENCE_LENGTH))
  model.add(Flatten())
  model.add(layers.Dense(10,  activation='relu'))
  model.add(layers.Dense(5, activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  model.summary()
  
  return model

---------
accuracy calculation function

عمل تابع لحساب الدقة

In [9]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

----------------------
**3- Feature Extraction:**  Word Embedding

استخراج الميزات(تحويل النص إلى أرقام)


**4- classification:** NN

تصنيف البيانات

with Cross validation

لتوزيع عينات الاختبار والتأكد من عدم وجود

Overfitting

In [10]:
%%time
from sklearn.model_selection import StratifiedKFold
from keras.backend import clear_session

# build train and test datasets
reviews = dataset['text'].values
sentiments = dataset['targe'].values
#####
strtfdKFold = StratifiedKFold(n_splits=3)#5
kfold = strtfdKFold.split(reviews, sentiments)
result_NNClassifier=[]
#
for k, (train_index, test_index) in enumerate(kfold):
    # if(os.path.exists("drive/MyDrive/mining/CNN.net")):
    #   print("load net")
    #   model = keras.models.load_model('drive/MyDrive/mining/CNN.net') 

    X_train_, X_test_ = reviews[train_index], reviews[test_index]
    y_train_, y_test_ = sentiments[train_index], sentiments[test_index]
    X_train, X_test ,y_train,y_test=Tokenizer_UNK(X_train_,X_test_,y_train_,y_test_)
    model=set_model()
    # Fit the model
    model.fit(X_train, y_train,epochs=2,verbose=True,batch_size=10)#validation_split=0.1,
    y_pred=(model.predict(X_test))
    y_classes_pre = [np.argmax(y, axis=None, out=None) for y in y_pred]
    y_classes_test = [np.argmax(y, axis=None, out=None) for y in y_test]
    clear_session()
    result_NNClassifier.append(calculate_results(y_classes_test, y_classes_pre))
    # model.save('drive/MyDrive/mining/CNN.net')
    # print("save net")

result_NNClassifier

Vocabulary size=325418
Number of Documents=68312
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 300)         97625400  
                                                                 
 flatten (Flatten)           (None, 300000)            0         
                                                                 
 dense (Dense)               (None, 10)                3000010   
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
Total params: 100,625,465
Trainable params: 100,625,465
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
4555/4555 [==============================] - 253s 55ms/step - loss: 0.1437 - accuracy: 0.8752
Epoch 2/2
4555/4555 [

حساب متوسط النتائج للـ5 مراحل من

 cross validation 

In [11]:
accuracy=0
f1=0
precision=0
recall=0
for i in result_NNClassifier:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(result_NNClassifier)
f1=f1/len(result_NNClassifier)
precision=precision/len(result_NNClassifier)
recall=recall/len(result_NNClassifier)
# (result_DecisionTreeClassifier)
results_NNClassifier=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
results_NNClassifier

[{'accuracy': 68.83746120129926,
  'f1': 0.6439032376678138,
  'precision': 0.6353495472360507,
  'recall': 0.6883746120129927}]

In [12]:
x=results_NNClassifier
f = open("NN_Em2.txt", "x")
with open('drive/MyDrive/mining/NN_Em2.txt', 'w') as f:
  for i in x:
    f.write('accuracy:'+str(i['accuracy']))
    f.write('\n')
    f.write('f1:'+str(i['f1']))
    f.write('\n')
    f.write('precision:'+str(i['precision']))
    f.write('\n')
    f.write('recall:'+str(i['recall']))
    f.write('\n')
    f.close()

--------
-----
---------
**5- Show Results**

النتائج

نتائج التدريب لكل قاعدة بيانات

In [ ]:
print('NNClassifier results in dataset 1:')
results_NNClassifier

NNClassifier results in dataset 1:


[{'accuracy': 95.44380449194637,
  'f1': 0.9544729877482121,
  'precision': 0.9548409360810294,
  'recall': 0.9544380449194636}]

In [ ]:
print('NNClassifier results in dataset 2:')
results_NNClassifier

NNClassifier results in dataset 2:


[{'accuracy': 81.30412192587461,
  'f1': 0.7834830431791028,
  'precision': 0.8486804632462394,
  'recall': 0.8130412192587462}]

In [ ]:
print('NNClassifier results in dataset 3:')
results_NNClassifier

NNClassifier results in dataset 3:


[{'accuracy': 74.74502511365579,
  'f1': 0.7144508172145367,
  'precision': 0.7094054648517151,
  'recall': 0.7474502511365579}]

In [13]:
print('NNClassifier results in dataset 4:')
results_NNClassifier

NNClassifier results in dataset 4:


[{'accuracy': 68.83746120129926,
  'f1': 0.6439032376678138,
  'precision': 0.6353495472360507,
  'recall': 0.6883746120129927}]

In [14]:
Final_Result=[
  {'accuracy': 95.44380449194637,
  'f1': 0.9544729877482121,
  'precision': 0.9548409360810294,
  'recall': 0.9544380449194636}
  ,
  {'accuracy': 81.30412192587461,
  'f1': 0.7834830431791028,
  'precision': 0.8486804632462394,
  'recall': 0.8130412192587462}
  ,
  {'accuracy': 74.74502511365579,
  'f1': 0.7144508172145367,
  'precision': 0.7094054648517151,
  'recall': 0.7474502511365579}
  ,
  {'accuracy': 68.83746120129926,
  'f1': 0.6439032376678138,
  'precision': 0.6353495472360507,
  'recall': 0.6883746120129927}
  ]

accuracy=0
f1=0
precision=0
recall=0
for i in Final_Result:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(Final_Result)
f1=f1/len(Final_Result)
precision=precision/len(Final_Result)
recall=recall/len(Final_Result)
# (result_DecisionTreeClassifier)
Final_Result=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
print("RAM: 1.7 ,GPU times: user 17min 55s ")
print('Final_Result:')
Final_Result

RAM: 1.7 ,GPU times: user 17min 55s 
Final_Result:


[{'accuracy': 80.08260318319401,
  'f1': 0.7740775214524164,
  'precision': 0.7870691028537586,
  'recall': 0.8008260318319401}]

---------------
